В данной тетрадке будут рассмотрены и применены различные DL подходы к решению проблемы "**Next-Basket Recommendation**" — генерация персонального ТОП-K списка «что купить дальше» на основе последовательности прошлых заказов.

Для решения данной проблемы будут использованы следующие архитектурные подходы:
1. GRU4Rec (baseline RNN)
2. SASRec — self-attention seq2seq
3. BERT4Rec — двусторонний Transformer с Cloze-маскировкой.

# Импорт библиотек

In [1]:
%pip uninstall -y numpy
%pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
%pip uninstall -y torch torchvision torchaudio
%pip install torch==2.5.1

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install --upgrade pyarrow pandas torchmetrics lightning recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 57.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently t

In [44]:
import os

import pandas as pd
import numpy as np
import torch, random
from torchmetrics.retrieval import RetrievalRecall, RetrievalNormalizedDCG
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd, pathlib, os

from recbole.quick_start import run_recbole
from recbole.config import Config
from recbole.model.sequential_recommender.gru4rec import GRU4Rec as RecboleGRU4Rec # Import Recbole's model class
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger, get_model, get_trainer

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def choose_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using GPU")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

In [5]:
#Все результаты моделей будем записывать в отдельный датафрэйм
df_res = []

# Импорт наших данных

In [6]:
data = pd.read_csv('InstaCart.csv')
data

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
0,16617,Organic Muenster Cheese Slices,21,16,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,8.0,0.0,dairy eggs,packaged cheese
1,21267,Sourdough Bread,112,3,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,3.0,0.0,bakery,bread
2,24561,Organic Cheese Frozen Pizza,79,1,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,1.0,0.0,frozen,frozen pizza
3,24852,Banana,24,4,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,2.0,0.0,produce,fresh fruits
4,27344,Uncured Genoa Salami,96,20,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,6.0,0.0,deli,lunch meat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51608,9434,Bag of Large Lemons,34,1,444424.0,206100.0,prior,1.0,3.0,13.0,0.0,1.0,0.0,frozen,frozen meat seafood
51609,39922,Celery Sticks,123,4,444424.0,206100.0,prior,1.0,3.0,13.0,0.0,2.0,0.0,produce,packaged vegetables fruits
51610,9434,Bag of Large Lemons,34,1,2483012.0,206100.0,prior,2.0,6.0,16.0,30.0,2.0,1.0,frozen,frozen meat seafood
51611,27156,Organic Black Beans,59,15,2483012.0,206100.0,prior,2.0,6.0,16.0,30.0,1.0,0.0,canned goods,canned meals beans


# Какую задачу решаем и что используем?

- Наша задача представляет из себя задачу рекомендации.
- В качестве baseline моделей мы будем использовать архитектурные подходы, которые будут работать только с последовательностями покупок без использования дополнительных признаков

# !!Теоретическая справка

В данном блоке будет кратко объяснено как работают использованные модели. Начнем с модели GRU4Rec

1. GRU4Rec
    - Данная модель основана на RNN (рекурентных нейронных сетях), для нашей задачи это значит, что последовательность покупок нашего пользователя может быть обработана, из нее могут быть "выцеплены" основные паттерны каждого из пользователей, что поможет сделать рекомендации лучше.

**Почему данная модель хороша?**
1. GRU-ячейка удерживает нужную память и забывает шум, поэтому "цепляет" зависимости даже из длинных послежовательностей без проблемы "затухающих градиентов".
2. Основана на последовательности, а не на мешке слов - модель воспринимает порядок покупок, а не просто набор покупок.
3. Эмбеддинги товаров обучаються вместе с RNN, то есть похожие товары сближаются в векторном пространстве и лучше обобщают на новые эмбеддинги новых товаров.
4. Pairwise-ranking loss (TOP1/BPR) учит скрытое состояние сразу расставлять top-K, а не просто классифицировать, что повышает Recall/NDCG.
5. Мини-батчи с негативным сэмплингом делают обучение быстрым (GPU) и масштабируемым до десятков миллионов взаимодействий.

# Предобработка данных

В рамках применяемой нами архитектуры нам нет нужды использовать все признаки, представленные в данных. Для нашей модели будет достаточно следующих признаков:
1. user_id
2. product_id
3. order_number
4. order_hour_of_day

Последние два признака будут использвоаны для того чтобы покупки пользователей могли бы выстроены в хронологическом порядке, в зависимости от времени покупки.

In [7]:
data.groupby('user_id')['product_id'].apply(list).reset_index(name="item_seq")

,user_id,item_seq
0,1097.0,"[16617, 21267, 24561, 24852, 27344, 38689, 423..."
1,1347.0,"[20940, 26348, 128, 13097, 13176, 21873, 22802..."
2,1519.0,"[47626, 207, 1006, 1463, 2009, 2481, 4825, 578..."
3,2047.0,"[3262, 4192, 19459, 21011, 23734, 24563, 29921..."
4,2876.0,"[495, 9551, 10312, 12821, 22120, 25013, 28199,..."
...,...,...
495,204342.0,"[1094, 3169, 5713, 7561, 7815, 16464, 20919, 2..."
496,204492.0,"[7781, 16668, 17794, 21841, 29095, 35750, 3602..."
497,205069.0,"[1405, 8843, 10673, 12817, 12916, 15739, 16997..."
498,205705.0,"[11782, 18232, 18465, 18523, 18703, 19508, 212..."


In [8]:

data.head()

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
0,16617,Organic Muenster Cheese Slices,21,16,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,8.0,0.0,dairy eggs,packaged cheese
1,21267,Sourdough Bread,112,3,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,3.0,0.0,bakery,bread
2,24561,Organic Cheese Frozen Pizza,79,1,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,1.0,0.0,frozen,frozen pizza
3,24852,Banana,24,4,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,2.0,0.0,produce,fresh fruits
4,27344,Uncured Genoa Salami,96,20,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,6.0,0.0,deli,lunch meat


In [9]:
features = ['user_id', 'product_id', 'order_number', 'order_hour_of_day']

df = data[features]
df.head()

,user_id,product_id,order_number,order_hour_of_day
0,1097.0,16617,1.0,14.0
1,1097.0,21267,1.0,14.0
2,1097.0,24561,1.0,14.0
3,1097.0,24852,1.0,14.0
4,1097.0,27344,1.0,14.0


In [10]:
#Создадим признак timestamp, который означает временую метку
df['timestamp'] = df['order_number'] * 24 + df['order_hour_of_day']
#отсортируем по user_id, timestamp
df = df.sort_values(['user_id', 'timestamp'])
df

/tmp/ipython-input-10-324218462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = df['order_number'] * 24 + df['order_hour_of_day']


,user_id,product_id,order_number,order_hour_of_day,timestamp
0,1097.0,16617,1.0,14.0,38.0
1,1097.0,21267,1.0,14.0,38.0
2,1097.0,24561,1.0,14.0,38.0
3,1097.0,24852,1.0,14.0,38.0
4,1097.0,27344,1.0,14.0,38.0
...,...,...,...,...,...
51608,206100.0,9434,1.0,13.0,37.0
51609,206100.0,39922,1.0,13.0,37.0
51610,206100.0,9434,2.0,16.0,64.0
51611,206100.0,27156,2.0,16.0,64.0


Теперь сформируем последовательность покупок для каждого пользователя

In [11]:
users_seq = df.groupby("user_id")["product_id"].apply(list).reset_index(name="seq")
users_seq

,user_id,seq
0,1097.0,"[16617, 21267, 24561, 24852, 27344, 38689, 423..."
1,1347.0,"[20940, 26348, 128, 13097, 13176, 21873, 22802..."
2,1519.0,"[47626, 207, 1006, 1463, 2009, 2481, 4825, 578..."
3,2047.0,"[3262, 4192, 19459, 21011, 23734, 24563, 29921..."
4,2876.0,"[495, 9551, 10312, 12821, 22120, 25013, 28199,..."
...,...,...
495,204342.0,"[1094, 3169, 5713, 7561, 7815, 16464, 20919, 2..."
496,204492.0,"[7781, 16668, 17794, 21841, 29095, 35750, 3602..."
497,205069.0,"[1405, 8843, 10673, 12817, 12916, 15739, 16997..."
498,205705.0,"[11782, 18232, 18465, 18523, 18703, 19508, 212..."


# Следующим этапом является токенизация и паддинг ID наших продуктов, так как на вход модель должна получать последовательность

In [12]:
PAD = 0
vocab = {PAD: "<PAD>"}

for pad in data['product_id'].unique():
    vocab[len(vocab)] = int(pad)

item2idx = {v:k for k,v in vocab.items()}

maxx = 100 #сколько покупок будет учитываться в модели

def encode(seq):
    ids = [item2idx[i] for i in seq][-maxx:]
    return [PAD] * (maxx - len(ids)) + ids

# Tran/Valid/Test Split

In [13]:
train, valid, test = [], [], []

for _, row in users_seq.iterrows():
    seq = row.seq
    if len(seq) < 3:
        continue

    train.append((encode(seq[:-2]), item2idx[seq[-2]]))
    valid.append((encode(seq[:-1]), item2idx[seq[-1]]))
    test .append((encode(seq),        item2idx[seq[-1]]))


# Pytorch Datasets

В данном разделе мы просто реализуем датасеты, с которыми работает Pytorch

In [14]:
class SeqDataset(torch.utils.data.Dataset):
    def __init__(self, pairs, num_items, train=True, neg_ratio=1):
        self.pairs, self.train, self.neg_ratio = pairs, train, neg_ratio
        self.num_items = num_items

    def __len__(self): return len(self.pairs)

    def __getitem__(self, idx):
        seq, pos = self.pairs[idx]
        seq = torch.tensor(seq, dtype=torch.long)
        if not self.train:
            return seq, torch.tensor(pos)
        negs = []
        while len(negs) < self.neg_ratio:
            n = random.randint(1, self.num_items-1)
            if n != pos: negs.append(n)
        return seq, torch.tensor(pos), torch.tensor(negs)


# GRU4Rec Baseline

В этой части тетрадки будет написана сама архитектура модели.

В GRU4Rec мы будем использовать специальный слой nn.GRU из Pytorch, который реализует слой LSTM из оригинальной статьи

In [15]:
import torch.nn as nn

import torch.nn as nn
import torch.nn.functional as F

class GRU4Rec(nn.Module):
    def __init__(self, num_items, d=128, hidden=256, num_layers=2, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_items, d, padding_idx=PAD)
        self.gru = nn.GRU(
            d,
            hidden,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.norm = nn.LayerNorm(hidden)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(hidden, num_items)

        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.out.weight)
        nn.init.zeros_(self.out.bias)
        for name, param in self.gru.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.zeros_(param.data)

    def forward(self, seq):
        x = self.dropout(self.emb(seq))

        x, _ = self.gru(x)

        h = x[:, -1, :]

        h = self.norm(h)
        h = self.dropout(h)

        return self.out(h)

# Training Loop
Напишем функцию обучения модели и протестируем на нашей архитектуре.

In [16]:
DEVICE = choose_device()
BATCH = 256
K = 20

train_ds = SeqDataset(train, len(vocab), train=True)
valid_ds = SeqDataset(valid, len(vocab), train=False)
test_ds = SeqDataset(test, len(vocab), train=False)

train_dl = torch.utils.data.DataLoader(train_ds, BATCH, shuffle=True, drop_last=True, pin_memory=True)
val_dl = torch.utils.data.DataLoader(valid_ds, BATCH, shuffle=False, pin_memory=True)
test_dl = torch.utils.data.DataLoader(test_ds, BATCH, shuffle=False, pin_memory=True)

model = GRU4Rec(len(vocab)).to(DEVICE)
opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
bce = nn.BCEWithLogitsLoss()

Using GPU


In [17]:
def train_loop(train_ld, valid_ld, DEVICE, K, BATCH, model, opt, bce):
    for epoch in range(50):
        torch.cuda.empty_cache()
        model.train()
        for seq, pos, neg in train_ld:
            seq, pos, neg = seq.to(DEVICE), pos.to(DEVICE), neg.to(DEVICE)
            logits = model(seq)
            pos_logits = logits.gather(1, pos.unsqueeze(1))
            neg_logits = logits.gather(1, neg)
            loss = bce(pos_logits, torch.ones_like(pos_logits)) + \
                bce(neg_logits, torch.zeros_like(neg_logits))
            opt.zero_grad(); loss.backward(); opt.step()

        model.eval()
        rec = RetrievalRecall(top_k=K).to(DEVICE)
        ndc = RetrievalNormalizedDCG(top_k=K).to(DEVICE)

        with torch.no_grad():
            for seq, pos in valid_ld:
                seq, pos = seq.to(DEVICE), pos.to(DEVICE)
                logits = model(seq)
                B, V = logits.shape

                target = torch.zeros_like(logits, dtype=torch.int)
                target[torch.arange(B), pos] = 1

                indexes = torch.arange(B, device=DEVICE).unsqueeze(1).expand(-1, V)

                rec.update(logits, target, indexes=indexes)
                ndc.update(logits, target, indexes=indexes)

        print(f"Epoch {epoch+1}  Recall@{K}: {rec.compute():.4f}  NDCG@{K}: {ndc.compute():.4f}")

In [18]:
seed_everything()
train_loop(train_dl, val_dl, DEVICE, K, BATCH, model, opt, bce)

Epoch 1  Recall@20: 0.0078  NDCG@20: 0.0034
Epoch 2  Recall@20: 0.0137  NDCG@20: 0.0090
Epoch 3  Recall@20: 0.0234  NDCG@20: 0.0160
Epoch 4  Recall@20: 0.0430  NDCG@20: 0.0250
Epoch 5  Recall@20: 0.0586  NDCG@20: 0.0333
Epoch 6  Recall@20: 0.0820  NDCG@20: 0.0462
Epoch 7  Recall@20: 0.0918  NDCG@20: 0.0546
Epoch 8  Recall@20: 0.0996  NDCG@20: 0.0623
Epoch 9  Recall@20: 0.1055  NDCG@20: 0.0660
Epoch 10  Recall@20: 0.1074  NDCG@20: 0.0697
Epoch 11  Recall@20: 0.1113  NDCG@20: 0.0705
Epoch 12  Recall@20: 0.1152  NDCG@20: 0.0710
Epoch 13  Recall@20: 0.1191  NDCG@20: 0.0704
Epoch 14  Recall@20: 0.1270  NDCG@20: 0.0735
Epoch 15  Recall@20: 0.1289  NDCG@20: 0.0740
Epoch 16  Recall@20: 0.1250  NDCG@20: 0.0727
Epoch 17  Recall@20: 0.1250  NDCG@20: 0.0729
Epoch 18  Recall@20: 0.1230  NDCG@20: 0.0715
Epoch 19  Recall@20: 0.1270  NDCG@20: 0.0737
Epoch 20  Recall@20: 0.1309  NDCG@20: 0.0752
Epoch 21  Recall@20: 0.1289  NDCG@20: 0.0743
Epoch 22  Recall@20: 0.1250  NDCG@20: 0.0738
Epoch 23  Recall@20

In [31]:
metrics_row = {}

model.eval()

gru4rec_pytorch_scores = []
true_items_pytorch = []

with torch.no_grad():
    for K in [5, 10, 20]:
        rec = RetrievalRecall(top_k=K).to(DEVICE)
        ndc = RetrievalNormalizedDCG(top_k=K).to(DEVICE)

        for seq, pos in test_dl:
            seq, pos = seq.to(DEVICE), pos.to(DEVICE)
            logits = model(seq)
            gru4rec_pytorch_scores.append(logits.cpu().numpy())
            true_items_pytorch.extend(pos.cpu().numpy())

            B, V = logits.shape
            target = torch.zeros_like(logits, dtype=torch.int)
            target[torch.arange(B), pos] = 1
            idx = torch.arange(B, device=DEVICE).unsqueeze(1).repeat(1, V)

            rec.update(logits.flatten(),  target.flatten(), indexes=idx.flatten())
            ndc.update(logits.flatten(),  target.flatten(), indexes=idx.flatten())

        metrics_row[f"recall@{K}"] = rec.compute().item()
        metrics_row[f"ndcg@{K}"]   = ndc.compute().item()
        print(f"K={K:2d}  Recall={metrics_row[f'recall@{K}']:.4f}  "
              f"NDCG={metrics_row[f'ndcg@{K}']:.4f}")

K= 5  Recall=0.0703  NDCG=0.0505
K=10  Recall=0.1152  NDCG=0.0687
K=20  Recall=0.1367  NDCG=0.0753


In [32]:
df_res =[]

df_res.append({
    'Model': 'GRU4Rec_PyTorch',
    'recall@5': metrics_row["recall@5"],
    'recall@10': metrics_row["recall@10"],
    'recall@20': metrics_row["recall@20"],
    'ndcg@5': metrics_row["ndcg@5"],
    'ndcg@10': metrics_row["ndcg@10"],
    'ndcg@20': metrics_row["ndcg@20"]
})
df_res

[{'Model': 'GRU4Rec_PyTorch',
  'recall@5': 0.0703125,
  'recall@10': 0.115234375,
  'recall@20': 0.13671875,
  'ndcg@5': 0.05049300193786621,
  'ndcg@10': 0.06865604966878891,
  'ndcg@20': 0.07531565427780151}]

# GRU4Rec from Recbole

Есть бибилотека recbole, созданная на основе Pytorch, для обучения рекомендательных алгоритмов. Я решил поробовать ей воспользовать и посмотреть на результаты.

In [36]:
df["user_id"] = df["user_id"].astype(int)
df["product_id"] = df["product_id"].astype(int)
df["timestamp"] = df["timestamp"].astype(int)

In [37]:
os.makedirs("recbole_data/instacart", exist_ok=True)
recbole_df = df[['user_id', 'product_id', 'timestamp']]

(recbole_df
  .rename(columns={
      "user_id":    "user_id:token",
      "product_id": "item_id:token",
      "timestamp":  "timestamp:float"
  })
  .to_csv("recbole_data/instacart/instacart.inter",
          sep="\t", index=False, header=True)
)

In [45]:
from recbole.quick_start import run_recbole
res_gru4 = run_recbole(config_file_list=['instacart.yaml'])

/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [39]:
df_res.append({
    'Model': 'GRU4Rec_from_scratch',
    'recall@5': res_gru4['test_result']['recall@5'],
    'recall@10': res_gru4['test_result']['recall@10'],
    'recall@20': res_gru4['test_result']['recall@20'],
    'ndcg@5': res_gru4['test_result']['ndcg@5'],
    'ndcg@10': res_gru4['test_result']['ndcg@10'],
    'ndcg@20': res_gru4['test_result']['ndcg@20']
})

pd.DataFrame(df_res)

,Model,recall@5,recall@10,recall@20,ndcg@5,ndcg@10,ndcg@20
0,GRU4Rec_PyTorch,0.070312,0.115234,0.136719,0.050493,0.068656,0.075316
1,GRU4Rec_from_scratch,0.166000,0.240000,0.310000,0.121800,0.146000,0.163800


# SASRec

SASRec — это один из более новых (2018 года) и сложных подходов, использующих механизм внимания. Этот подход позволяет выделять сложные закономерности подобно RNN-моделям, но снизить требования к объемам обучающей выборки.

На каждом шаге SASRec пытается определить, какие действия являются наиболее важными из истории пользователя, и использовать их для прогнозирования следующего действия. еханизм self-attention, схема которого приведена ниже, отлично зарекомендовал себя в задачах описания изображений, обобщения текста, машинном переводе. В некотором смысле, эти задачи похожи на рекомендации.

![SASRec](https://habrastorage.org/r/w1560/getpro/habr/upload_files/718/806/b35/718806b3506f05814e85c31a2e647d41.png)

In [58]:
from recbole.quick_start import run_recbole
res = run_recbole(config_file_list=['instacart_SASRec.yaml'])

/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [59]:
df_res.append({
    'Model': 'SASRec_from_scratch',
    'recall@5': res['test_result']['recall@5'],
    'recall@10': res['test_result']['recall@10'],
    'recall@20': res['test_result']['recall@20'],
    'ndcg@5': res['test_result']['ndcg@5'],
    'ndcg@10': res['test_result']['ndcg@10'],
    'ndcg@20': res['test_result']['ndcg@20']
})

pd.DataFrame(df_res)

,Model,recall@5,recall@10,recall@20,ndcg@5,ndcg@10,ndcg@20
0,GRU4Rec_PyTorch,0.070312,0.115234,0.136719,0.050493,0.068656,0.075316
1,GRU4Rec_from_scratch,0.166000,0.240000,0.310000,0.121800,0.146000,0.163800
2,SASRec_from_scratch,0.268000,0.336000,0.406000,0.184300,0.206200,0.224200


Ранее мы применили SASRec к данным без дополнительных признаков, только на последовательность покупок, но что будет если добавить новые признаки?

Далее представлен код, в котором помимо уже существующих признаков, добавлены новые фичи, такие как "aisle_id", "days_since_prior_order". Смогут ли эти признаки улучшить качество рекомендаций?

In [ ]:
data = pd.read_csv("InstaCart.csv")
item_feat = (
    data[["product_id", "aisle_id"]]
    .drop_duplicates()
    .rename(columns={"product_id": "item_id"})
)

out_dir = pathlib.Path("recbole_data/instacart_ext")
item_feat.to_csv(
    out_dir / "instacart_ext.item",
    sep="\t",
    index=False,
    header=["item_id:token", "aisle_id:token"],
)

item_feat

In [ ]:
data["dspo"] = data["days_since_prior_order"].fillna(0).astype(int).astype(str)
data['timestamp'] = data['order_number'] * 24 + data['order_hour_of_day']
seq = (
    data.sort_values(["user_id", "timestamp"])
      .groupby("user_id")["dspo"]
      .apply(" ".join)
      .reset_index()
)

inter = data[["user_id", "product_id", "timestamp"]].rename(columns={
    "product_id": "item_id"
})

inter = inter.merge(seq, on="user_id")

out = pathlib.Path("recbole_data/instacart_ext/instacart_ext.inter")
inter.to_csv(
    out, sep="\t", index=False,
    header=[
        "user_id:token",
        "item_id:token",
        "timestamp:float",
        "days_since_prior_order_list:float_seq"
    ]
)
seq

,user_id,dspo
0,1097.0,0 0 0 0 0 0 0 0 28 28 28 28 28 28 28 28 30 30 30
1,1347.0,0 0 30 30 30 30 30 30 30 10 21 21 7 7 23 23 23...
2,1519.0,5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 ...
3,2047.0,0 0 0 0 0 0 0 0 0 0 0 0 30 30 30 30 30 30 30 3...
4,2876.0,0 0 0 0 0 0 0 0 0 0 0 0 9 9 9 9 9 9 9 5 5 5 7 ...
...,...,...
495,204342.0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 7 7 7 7 7 7 ...
496,204492.0,0 0 0 0 0 0 0 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 ...
497,205069.0,0 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 30 3...
498,205705.0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 30 30 30 30 30...


In [ ]:
res = run_recbole(config_file_list=['instacart_SASRec_external.yaml'])

ValueError: Neither [./recbole_data/instacart_ext] exists in the device nor [instacart_ext] a known dataset name.

Данный код должен работать, но из-за ограниченности ресурсов, возникает проблема, которая ведет к перезапуску системы.

#Bert4Rec

Заключительным штрихом в нашей работе станет модель Bert4Rec.

Данная модель похожа на предыдущую использование механизма внимания в своей архитектуре. Главное отличие, что модель смотрит на последовательность покупок не только слева направо, но и с другой стороны.

Рисерчеры из Alibaba в 2019 году впервые подружили BERT-ы и Recsys, представив архитектуру BERT4Rec. Вообще, идея логично проистекает из SASRec - если модели с однонаправленным контекстом (unidirectional models) отлично справились с бенчмарками Recsys, почему бы не поставить эксперимент над моделями с двунаправленными контекстом (bidirectional models).

Таким образом можно:

Обусловить item-ы контекстом справа, получив их лучшее представление, и заодно значительно увеличить обучающую выборку (комбинаторный взрыв количества перестановок маскированных item-ов и немаскированных).

Смягчить предположение о жесткой упорядоченности item-ов в последовательности, которая соблюдается не всегда.

In [60]:
res = run_recbole(config_file_list=['instacart_BERT4Rec.yaml'])

/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [61]:
df_res.append({
    'Model': 'BERT4Rec_from_scratch',
    'recall@5': res['test_result']['recall@5'],
    'recall@10': res['test_result']['recall@10'],
    'recall@20': res['test_result']['recall@20'],
    'ndcg@5': res['test_result']['ndcg@5'],
    'ndcg@10': res['test_result']['ndcg@10'],
    'ndcg@20': res['test_result']['ndcg@20']
})

pd.DataFrame(df_res)

,Model,recall@5,recall@10,recall@20,ndcg@5,ndcg@10,ndcg@20
0,GRU4Rec_PyTorch,0.070312,0.115234,0.136719,0.050493,0.068656,0.075316
1,GRU4Rec_from_scratch,0.166000,0.240000,0.310000,0.121800,0.146000,0.163800
2,SASRec_from_scratch,0.268000,0.336000,0.406000,0.184300,0.206200,0.224200
3,BERT4Rec_from_scratch,0.096000,0.130000,0.188000,0.058400,0.069100,0.083700


# Выводы

По результатам сравнения моделей для задачи рекомендаций следующей покупки (Next-Basket Recommendation) на данных Instacart, наилучшую производительность показала модель **SASRec**, существенно опередив GRU4Rec и BERT4Rec по всем метрикам (Recall\@K, NDCG\@K).

Причины такого превосходства заключаются в следующем:

1. **Использование self-attention**:

   * SASRec строит представления товаров с учётом всех позиций в последовательности благодаря механизму self-attention, что позволяет более эффективно учитывать долгосрочные зависимости в поведении пользователя, чем рекуррентные подходы (GRU4Rec).

2. **Односторонняя направленность attention**:

   * В отличие от BERT4Rec, который использует двунаправленное внимание, SASRec целенаправленно учитывает последовательность покупок с фокусом на будущие события, что в контексте задачи рекомендаций оказалось эффективнее.

3. **Эффективность обучения и стабильность результатов**:

   * SASRec требует меньших вычислительных ресурсов для обучения и быстрее сходится к оптимальным результатам по сравнению с более сложными архитектурами (BERT4Rec), что облегчает экспериментирование и подбор гиперпараметров.

Таким образом, использование SASRec рекомендуется для задачи генерации персонализированных рекомендаций следующей покупки в Instacart, что позволит повысить точность рекомендаций, увеличить конверсию и, в конечном счёте, позитивно повлиять на бизнес-метрики (средний чек, повторные покупки, LTV).
